In [2]:
!pip install pandas-summary ipython fastai==0.7
# !mkdir data/bulldozer

    100% |████████████████████████████████| 122kB 3.7MB/s ta 0:00:01
    100% |████████████████████████████████| 496.4MB 88kB/s  eta 0:00:01  9% |███                             | 45.6MB 36.6MB/s eta 0:00:13��▌                          | 85.3MB 13.7MB/s eta 0:00:30    24% |████████                        | 122.5MB 32.8MB/s eta 0:00:12    30% |█████████▉                      | 152.8MB 36.5MB/s eta 0:00:10    35% |███████████▎                    | 174.4MB 31.9MB/s eta 0:00:11    37% |████████████                    | 186.0MB 34.6MB/s eta 0:00:09    47% |███████████████▏                | 235.7MB 27.9MB/s eta 0:00:10    48% |███████████████▍                | 238.3MB 7.6MB/s eta 0:00:35    48% |███████████████▍                | 238.9MB 18.4MB/s eta 0:00:14    53% |█████████████████▏              | 265.6MB 28.7MB/s eta 0:00:09    57% |██████████████████▎             | 284.0MB 34.7MB/s eta 0:00:07    59% |███████████████████             | 293.6MB 32.8MB/s eta 0:00:07    60% |█████████████████

  Found existing installation: torch 1.0.0
    Uninstalling torch-1.0.0:
      Successfully uninstalled torch-1.0.0
  Found existing installation: fastai 1.0.39
    Uninstalling fastai-1.0.39:
      Successfully uninstalled fastai-1.0.39
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import IPython

from sklearn import metrics

In [7]:
PATH = 'data/bulldozer/'
# Kaggle Kernel
# !mkdir data && cp -r ../input/train data/bulldozer
# !ls data/bulldozer -a

.  ..  Train.csv


In [38]:
# df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])
df_raw = pd.read_feather('tmp/bulldozers-raw')

/opt/conda/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [39]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
display_all(df_raw.tail().transpose())

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,9.25913,9.30565,9.3501,9.10498,8.95545
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,-1,-1,-1,-1,-1
fiModelDesc,35NX2,35NX2,35NX2,30NX,30NX


In [10]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [11]:
# df_raw.saledate.head
add_datepart(df_raw, 'saledate')

In [12]:
# df_raw.saleYear.head()
# display_all(df_raw.tail().transpose())
train_cats(df_raw)
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
df_raw.UsageBand = df_raw.UsageBand.cat.codes

In [13]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [14]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

In [40]:
x, y, nas = proc_df(df_raw, 'SalePrice')

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x, y)
m.score(x, y)

In [16]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()
n_valid = 12000
n_trn = len(x) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(x, n_trn)
y_train, y_valid = split_vals(y, n_trn)
X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

In [17]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [41]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x, y)
m.score(x, y)
print_score(m)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1min 35s, sys: 576 ms, total: 1min 36s
Wall time: 50 s
[7.833346804364986, 0.09402240646118123, -13846.111600969298, 0.984212596857716]


In [19]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [42]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [23]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 3.82 s, sys: 12 ms, total: 3.83 s
Wall time: 2.12 s
[0.011130732176392277, 7.763913085388849, 0.9720416234555509, -106.64882749869577]


In [43]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.5211497770627427, 0.5837391569223966, 0.4113438446206489, 0.39146490856702276]


In [35]:
draw_tree(m.estimators_[0], df_trn, precision=3)

NameError: name 'IPython' is not defined

In [36]:
?IPython

In [44]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[3.7682219008410604e-17, 0.47834235871776437, 1.0, 0.5913743208353788]


In [45]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.10979170295843048, 0.3587331207651305, 0.9738738377381285, 0.7701783569047287]


In [47]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

(array([10.04324949,  9.51044496,  9.51044496,  9.54681261,  9.43348392,
         9.30565055,  9.47270464,  8.9226583 ,  9.10497986,  9.15904708]),
 9.40094763773077,
 9.104979856318357)

In [48]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, 
                          oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.09489419380349602, 0.350410830906491, 0.9804828743540394, 0.7807179832640307, 0.8591570059240375]


In [50]:
df_trn, y_trn = proc_df(df_raw, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
set_rf_samples(20000)

ValueError: too many values to unpack (expected 2)